# Financial Analysis Multi Agent System

This Multi Agent System analyzes market data to provide trading insights and evaluate the risks on them. It consists of 4 agents: Data Analyser, Strategy Developer, 

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [ ]:
import boto3
import json
import os

# Function to get secret from AWS Secrets Manager
def get_secret(secret_name, region_name="ap-southeast-1"):
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except Exception as e:
        raise e

    if 'SecretString' in get_secret_value_response:
        return json.loads(get_secret_value_response['SecretString'])
    else:
        return get_secret_value_response['SecretBinary'].decode('utf-8')

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
def load_env():
    _ = load_dotenv(find_dotenv())

try:
    openai_secrets = get_secret("crewai-openai-key") 
    os.environ["OPENAI_API_KEY"] = openai_secrets["OPENAI_API_KEY"]
except Exception as e:
    print(f"Error retrieving OpenAI secret: {e}")
    # Fallback or error handling

try:
    serper_secrets = get_secret("crewai-serper-key") 
    os.environ["SERPER_API_KEY"] = serper_secrets["SERPER_API_KEY"]
except Exception as e:
    print(f"Error retrieving Serper secret: {e}")
    # Fallback or error handling


# break line every 80 characters if line is longer than 80 characters
# don't break in the middle of a word
def pretty_print_result(result):
  parsed_result = []
  for line in result.split('\n'):
      if len(line) > 80:
          words = line.split(' ')
          new_line = ''
          for word in words:
              if len(new_line) + len(word) + 1 > 80:
                  parsed_result.append(new_line)
                  new_line = word
              else:
                  if new_line == '':
                      new_line = word
                  else:
                      new_line += ' ' + word
          parsed_result.append(new_line)
      else:
          parsed_result.append(line)
  return "\n".join(parsed_result)


In [3]:
import os
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Tools

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [7]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [11]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [12]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [13]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [14]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew

In [15]:
from crewai import Crew, Process
from langchain_community.llms import Bedrock

bedrock_llm = Bedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name="ap-southeast-1",
    model_kwargs={"temperature": 0.7} 
)

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=bedrock_llm,
    process=Process.hierarchical, # perform heirarchily
    verbose=True
)

## Running the Crew


In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [17]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: To effectively monitor and analyze market data for AAPL, I need to leverage the expertise of my team. I should first delegate the analysis of historical and real-time market data to the Data Analyst, who can use statistical modeling and machine learning to identify trends and predict future market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Analyze market data", "context": "Use historical and real-time data for AAPL to apply statistical modeling and machine learning techniques. Identify trends, patterns, and potential predictive indicators that could inform about future market movements of AAPL. Provide a detailed report on findings, including any sig

Thought: I have successfully retrieved detailed historical data for AAPL from Yahoo Finance. This data includes daily open, high, low, close prices, and volume from July 30, 2024, to July 30, 2025. This comprehensive dataset is critical for performing time-series analysis to identify trends and make predictions about future stock movements. I can now use this information to apply statistical modeling and machine learning techniques to identify patterns and predictive indicators.

Final Answer: The historical data for AAPL from July 30, 2024, to July 30, 2025, includes detailed daily stock movements with open, high, low, close prices, and volume. This data is essential for conducting a thorough analysis to identify trends and predict future movements in the AAPL stock. By applying statistical and machine learning methods to this data, we can extract significant trends and predictive indicators that will inform future trading decisions. This analysis can reveal potential market opportuni

Thought: I now know the final answer based on the comprehensive analysis and strategic development provided by my team. The Data Analyst has analyzed the historical and real-time market data for AAPL, identifying significant trends and predictive indicators. The Trading Strategy Developer has then used these insights to develop robust trading strategies. These strategies are designed to maximize returns and minimize risks by adapting to both short-term fluctuations and long-term trends. The strategy includes using technical indicators like SMA, RSI, MACD, and ATR along with Fibonacci retracement levels for effective trading decisions. Continuous monitoring and adjustment are recommended to adapt to changing market conditions.

Final Answer: Insights and alerts about significant market opportunities or threats for AAPL include:

1. **Significant Trends and Predictive Indicators**:
   - Long-term bullish trends when AAPL's price crosses above the 200-day SMA.
   - Potential bearish trend

Thought: The search results highlight key aspects of day trading, such as the need for significant startup capital, thorough knowledge of market strategies, and the ability to make quick decisions based on short-term price fluctuations. This information will help in adapting our existing strategies for AAPL to suit a day trading format. I need to consider how to modify each of our existing strategies to fit into the day trading framework, ensuring they can handle rapid entries and exits and manage risk effectively.

Final Answer: To adapt our existing trading strategies for AAPL to fit a day trading format while maintaining a medium risk tolerance, we need to modify each strategy with a focus on short-term movements and quicker trade execution. Here's how we can adjust each strategy:

1. **Trend Following with SMA 200**: Since day trading involves a much shorter time horizon, using a 200-day simple moving average (SMA) is not practical. We should instead consider shorter periods like a

Thought: The search results provided several relevant links that potentially contain detailed analyses on using SMA, RSI, MACD, ATR, and Fibonacci retracement for day trading AAPL. I will start by exploring the article from TradeSearcher which seems to focus on trading strategies including Fibonacci retracement and VWAP, which could be insightful for refining our strategies.

Action: Read website content
Action Input: {"website_url": "https://tradesearcher.ai/symbols/market/stock/384-aapl"} 

Top 50 Apple Inc. (AAPL) Trading Strategies in 2025 - TradeSearcherTradeSearcherTrading Strategies Free Tools Search Strategies for Bitcoin, AAPL, EUR/USD...Search Strategies for Bitcoin, AAPL, EUR/USD...Home›Symbols›Market›Stock›384 AaplBest Trading Strategies for Apple Inc. (AAPL) You are currently in the Demo Mode with limited data, to access all strategies please create your accountFiltersSort ByRandomSort byTTP Intelligent AccumulatorApple Inc. (AAPL)@ 15 minDetailsTradingView 2.25Risk Reward

...

Thought: I now know the final answer

Final Answer: Based on the current market conditions for AAPL, including the recent price fluctuations and trading volumes, here are the recommended applications of our trading strategies to optimize execution:

1. **Short-Term Trend Following with SMA**: Given the recent price data, applying a short-term Simple Moving Average (SMA) can help in identifying the current market trend. The use of a 10-day SMA might be suitable to capture short-term movements and generate buy or sell signals accordingly.

2. **Momentum Trading with Adjusted RSI and MACD**: The Relative Strength Index (RSI) and Moving Average Convergence Divergence (MACD) can be adjusted to reflect shorter time frames, aligning with the daily trading volume and price changes. Considering the volatility, a tighter MACD setting could be more responsive to price movements, and an RSI level adjusted to 30/70 (instead of the standard 20/80) might better indicate overbought or oversold conditi

Thought: To provide a comprehensive risk analysis for the proposed trading strategies for AAPL, I need to understand the specific risks associated with each strategy and the overall execution plan. It will be essential to consider market volatility, the effectiveness of the technical indicators proposed, and the potential for significant financial losses. I will start by consulting with the Risk Advisor to get an initial assessment of the general and specific risks linked to these strategies.

Action: Ask question to co-worker
Action Input: {"coworker": "Risk Advisor", "question": "What are the potential risks associated with the proposed trading strategies for AAPL, including Short-Term Trend Following with SMA, Momentum Trading with Adjusted RSI and MACD, Volatility-Based Position Sizing with Short-Term ATR, Intraday Fibonacci Retracement, and Continuous Monitoring and Dynamic Adjustment? Please provide a detailed analysis of each strategy and suggest possible mitigation strategies."

Thought: The search results have provided insights into the risks associated with Momentum Trading using Adjusted RSI and MACD. These risks include frequent false signals, especially in volatile or sideways markets, which can lead to frequent trades and potential losses. There is also a risk of over-reliance on indicators without adequate risk management. Now, I need to explore the risks associated with Volatility-Based Position Sizing using Short-Term ATR.

Action: Search the internet
Action Input: {"search_query": "risks of Volatility-Based Position Sizing with Short-Term ATR"} 


Search results: Title: Volatility-Based Position Sizing - QuantifiedStrategies.com
Link: https://www.quantifiedstrategies.com/volatility-based-position-sizing/
Snippet: Volatility-based position sizing allows traders to adjust their capital allocation according to market volatility, thus managing risk more effectively.
---
Title: How To Reduce Risk With Optimal Position Size - Investopedia
Link: https://www

Action Input: {"search_query": "risks of Continuous Monitoring and Dynamic Adjustment in trading"} 


Search results: Title: The Importance of Continuous Monitoring in Financial Risk ...
Link: https://www.wrightresearch.in/blog/the-importance-of-continuous-monitoring-in-financial-risk-management-for-investments/
Snippet: Continuous monitoring in investments acts as an early warning system, helping you identify and manage risks before they impact your ...
---
Title: Importance of Risk Management in Algo Trading - uTrade Algos
Link: https://www.utradealgos.com/blog/risk-management-in-algo-trading
Snippet: Risk management involves continuous monitoring and adjustment of algorithms to prevent overfitting and mitigate the influence of data biases. Human Errors ...
---
Title: 5 Stock Options Data Strategies for Managing Risk in Trading - Intrinio
Link: https://intrinio.com/blog/risk-management-in-options-trading
Snippet: Continuously Monitor and Adjust Your Portfolio. Markets are constantly 

Thought: I now know the final answer

Final Answer: The comprehensive risk analysis report for the proposed trading strategies for AAPL includes the following details:

1. **Short-Term Trend Following with SMA**:
   - **Risks**: Overtrading during volatile periods, potential for incorrect trend identification, and significant equity drawdowns if market conditions change abruptly.
   - **Mitigation**: Implement additional filters to confirm trends, employ stop-loss orders to limit losses, and diversify trading across different time frames and assets.

2. **Momentum Trading with Adjusted RSI and MACD**:
   - **Risks**: Frequent false signals in volatile or sideways markets, leading to frequent trades and potential losses.
   - **Mitigation**: Combine these indicators with other forms of analysis (e.g., fundamental analysis), use tight stop-losses to control losses, and ensure disciplined risk management by only risking a small percentage of capital on each trade.

3. **Volatility-Based P

In [18]:
from IPython.display import Markdown
Markdown(result)

The comprehensive risk analysis report for the proposed trading strategies for AAPL includes the following details:

1. **Short-Term Trend Following with SMA**:
   - **Risks**: Overtrading during volatile periods, potential for incorrect trend identification, and significant equity drawdowns if market conditions change abruptly.
   - **Mitigation**: Implement additional filters to confirm trends, employ stop-loss orders to limit losses, and diversify trading across different time frames and assets.

2. **Momentum Trading with Adjusted RSI and MACD**:
   - **Risks**: Frequent false signals in volatile or sideways markets, leading to frequent trades and potential losses.
   - **Mitigation**: Combine these indicators with other forms of analysis (e.g., fundamental analysis), use tight stop-losses to control losses, and ensure disciplined risk management by only risking a small percentage of capital on each trade.

3. **Volatility-Based Position Sizing with Short-Term ATR**:
   - **Risks**: Being too conservative can result in missed opportunities, while being too aggressive can lead to significant financial losses.
   - **Mitigation**: Regularly adjust risk parameters based on ongoing market analysis and backtesting to find the optimal balance between risk and reward.

4. **Intraday Fibonacci Retracement**:
   - **Risks**: Potential for false signals and over-reliance on Fibonacci levels alone for trading decisions; volatility of markets can affect the effectiveness of the tool.
   - **Mitigation**: Use Fibonacci levels in conjunction with other technical indicators and patterns. Implement a comprehensive risk management strategy including predefined entry and exit rules.

5. **Continuous Monitoring and Dynamic Adjustment**:
   - **Risks**: Overfitting, data biases, human errors in managing algorithms, and technical failures.
   - **Mitigation**: Establish robust protocols for algorithm management and maintenance, including frequent updates and backtests to adapt to market conditions. Implement fail-safes and redundancy plans to handle technical failures.

This report outlines the potential risks associated with each trading strategy and provides a robust set of mitigation strategies to address these risks effectively.